<a href="https://colab.research.google.com/github/witold87/pub_haystack_unstructurized/blob/develop/unstructured.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
! pip install farm-haystack==0.8.0

     |████████████████████████████████| 152 kB 5.0 MB/s 
     |████████████████████████████████| 203 kB 60.5 MB/s 
     |████████████████████████████████| 52 kB 1.1 MB/s 
     |████████████████████████████████| 54 kB 3.4 MB/s 
     |████████████████████████████████| 79 kB 11.7 MB/s 
     |████████████████████████████████| 321 kB 74.4 MB/s 
     |████████████████████████████████| 359 kB 73.9 MB/s 
     |████████████████████████████████| 85 kB 6.5 MB/s 
     |████████████████████████████████| 981 kB 51.5 MB/s 
     |████████████████████████████████| 5.6 MB 52.6 MB/s 
     |████████████████████████████████| 100 kB 13.4 MB/s 
     |████████████████████████████████| 8.6 MB 21.3 MB/s 
     |████████████████████████████████| 402 kB 75.2 MB/s 
     |████████████████████████████████| 56 kB 5.9 MB/s 
     |████████████████████████████████| 3.0 MB 58.7 MB/s 
     |████████████████████████████████| 3.8 MB 51.9 MB/s 
     |████████████████████████████████| 776.8 MB 17 kB/s 
     |██████████████████

In [2]:
import logging

from haystack.document_store.memory import InMemoryDocumentStore
from haystack.document_store.faiss import FAISSDocumentStore
from haystack.document_store.elasticsearch import ElasticsearchDocumentStore
#from haystack.preprocessor.utils import convert_files_to_dicts, fetch_archive_from_http
from haystack.reader import TransformersReader
from haystack.reader import FARMReader
from haystack.retriever import DensePassageRetriever
from haystack.retriever import ElasticsearchRetriever
from haystack.utils import print_answers, launch_es
from haystack.preprocessor.cleaning import clean_wiki_text


01/21/2022 10:10:09 - INFO - faiss.loader -   Loading faiss with AVX2 support.
01/21/2022 10:10:09 - INFO - faiss.loader -   Could not load library with AVX2 support due to:
ModuleNotFoundError("No module named 'faiss.swigfaiss_avx2'")
01/21/2022 10:10:09 - INFO - faiss.loader -   Loading faiss.
01/21/2022 10:10:09 - INFO - faiss.loader -   Successfully loaded faiss.
01/21/2022 10:10:10 - INFO - farm.modeling.prediction_head -   Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .


In [3]:
reader = FARMReader(model_name_or_path="deepset/xlm-roberta-large-squad2", use_gpu=False)

01/21/2022 10:10:15 - INFO - farm.utils -   Using device: CPU 
01/21/2022 10:10:15 - INFO - farm.utils -   Number of GPUs: 0
01/21/2022 10:10:15 - INFO - farm.utils -   Distributed Training: False
01/21/2022 10:10:15 - INFO - farm.utils -   Automatic Mixed Precision: None
01/21/2022 10:11:14 - WARNING - farm.utils -   ML Logging is turned off. No parameters, metrics or artifacts will be logged to MLFlow.
01/21/2022 10:11:14 - INFO - farm.utils -   Using device: CPU 
01/21/2022 10:11:14 - INFO - farm.utils -   Number of GPUs: 0
01/21/2022 10:11:14 - INFO - farm.utils -   Distributed Training: False
01/21/2022 10:11:14 - INFO - farm.utils -   Automatic Mixed Precision: None
01/21/2022 10:11:15 - INFO - farm.infer -   Got ya 2 parallel workers to do inference ...
01/21/2022 10:11:15 - INFO - farm.infer -    0    0 
01/21/2022 10:11:15 - INFO - farm.infer -   /w\  /w\
01/21/2022 10:11:15 - INFO - farm.infer -   /'\  / \
01/21/2022 10:11:15 - INFO - farm.infer -     


In [4]:
# ## Task: Question Answering for Game of Thrones
#
# Question Answering can be used in a variety of use cases. A very common one:  Using it to navigate through complex
# knowledge bases or long documents ("search setting").
#
# A "knowledge base" could for example be your website, an internal wiki or a collection of financial reports.
# In this tutorial we will work on a slightly different domain: "Game of Thrones".
#
# Let's see how we can use a bunch of Wikipedia articles to answer a variety of questions about the
# marvellous seven kingdoms.

import logging

from haystack.document_store.memory import InMemoryDocumentStore
from haystack.document_store.faiss import FAISSDocumentStore
from haystack.document_store.elasticsearch import ElasticsearchDocumentStore
#from haystack.preprocessor.utils import convert_files_to_dicts, fetch_archive_from_http
from haystack.reader import TransformersReader
from haystack.reader import FARMReader
from haystack.retriever import DensePassageRetriever
from haystack.retriever import ElasticsearchRetriever
from haystack.utils import print_answers, launch_es
from haystack.preprocessor.cleaning import clean_wiki_text
#from memory_profiler import profile

def tutorial1_basic_qa_pipeline():
    logger = logging.getLogger(__name__)

    # ## Document Store
    #
    # Haystack finds answers to queries within the documents stored in a `DocumentStore`. The current implementations of
    # `DocumentStore` include `ElasticsearchDocumentStore`, `FAISSDocumentStore`, `SQLDocumentStore`, and `InMemoryDocumentStore`.
    #
    # **Here:** We recommended Elasticsearch as it comes preloaded with features like full-text queries, BM25 retrieval,
    # and vector storage for text embeddings.
    # **Alternatives:** If you are unable to setup an Elasticsearch instance, then follow the Tutorial 3
    # for using SQL/InMemory document stores.
    # **Hint**:
    # This tutorial creates a new document store instance with Wikipedia articles on Game of Thrones. However, you can
    # configure Haystack to work with your existing document stores.
    #
    # Start an Elasticsearch server
    # You can start Elasticsearch on your local machine instance using Docker. If Docker is not readily available in
    # your environment (e.g. in Colab notebooks), then you can manually download and execute Elasticsearch from source.

    #launch_es()

    # Connect to Elasticsearch
    #document_store = ElasticsearchDocumentStore(host="localhost", username="", password="", index="document")
    document_store = FAISSDocumentStore(faiss_index_factory_str='Flat')

    # ## Preprocessing of documents
    #
    # Haystack provides a customizable pipeline for:
    # - converting files into texts
    # - cleaning texts
    # - splitting texts
    # - writing them to a Document Store

    # In this tutorial, we download Wikipedia articles about Game of Thrones, apply a basic cleaning function, and add
    # them in Elasticsearch.

    # Let's first fetch some documents that we want to query
    # Here: 517 Wikipedia articles for Game of Thrones
    #doc_dir = "data/article_txt_got"
    #s3_url = "https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/wiki_gameofthrones_txt.zip"
    #fetch_archive_from_http(url=s3_url, output_dir=doc_dir)

    # convert files to dicts containing documents that can be indexed to our datastore
    # dicts = convert_files_to_dicts(dir_path=doc_dir, split_paragraphs=True)
    #dicts = convert_files_to_dicts(dir_path=doc_dir, clean_func=clean_wiki_text, split_paragraphs=True)
    # You can optionally supply a cleaning function that is applied to each doc (e.g. to remove footers)
    # It must take a str as input, and return a str.
    # dicts = [{'text':
    #           """Lululemon Athletica Inc. (CA) started trading on May 2, 2007, operates in the Consumer Cyclical sector (Apparel Retail industry), has 25,000 full-time employees, and is led by Mr. Calvin McDonald. lululemon athletica inc., together with its subsidiaries, designs, distributes, and retails athletic apparel and accessories for women and men. It operates through two segments, Company-Operated Stores and Direct to Consumer. The company offers pants, shorts, tops, and jackets for healthy lifestyle and athletic activities, such as yoga, running, and training, as well as other sweaty pursuits. It also provides fitness-related accessories. The company sells its products through a chain of company-operated stores; outlets and warehouse sales; a network of wholesale accounts, such as yoga studios, health clubs, and fitness centers; temporary locations, including seasonal stores; and license and supply arrangements, as well as directly to consumer through mobile apps, and lululemon.com e-commerce website. As of January 31, 2021, it operated 521 company-operated stores under the lululemon brand in the United States, Canada, the People's Republic of China, Australia, the United Kingdom, Japan, New Zealand, Germany, South Korea, Singapore, France, Malaysia, Sweden, Ireland, the Netherlands, Norway, and Switzerland. lululemon athletica inc. was founded in 1998 and is based in Vancouver, Canada.
    #           """,
    #           'meta': {'name': 'lulu.txt'}}]
    # print(f'{dicts[0]}')

    with open('nestle_report_de.txt', 'r') as file:
        text = file.readlines()

    text_str = '\n'.join([str(elem) for elem in text])
    stripped_text = text_str.strip('\n')
    print(stripped_text)

    dicts = [{'text': stripped_text,
              'meta': {'name': 'thai_air_th.txt'}}]

    document_store.write_documents(dicts)

    # ## Initalize Retriever & Reader
    #
    # ### Retriever
    #
    # Retrievers help narrowing down the scope for the Reader to smaller units of text where a given question
    # could be answered.
    #
    # They use some simple but fast algorithm.
    # **Here:** We use Elasticsearch's default BM25 algorithm
    # **Alternatives:**
    # - Customize the `ElasticsearchRetriever`with custom queries (e.g. boosting) and filters
    # - Use `EmbeddingRetriever` to find candidate documents based on the similarity of
    #   embeddings (e.g. created via Sentence-BERT)
    # - Use `TfidfRetriever` in combination with a SQL or InMemory Document store for simple prototyping and debugging

    retriever = DensePassageRetriever(document_store=document_store)

    document_store.update_embeddings(retriever)
    # Alternative: An in-memory TfidfRetriever based on Pandas dataframes for building quick-prototypes
    # with SQLite document store.
    #
    # from haystack.retriever.tfidf import TfidfRetriever
    # retriever = TfidfRetriever(document_store=document_store)

    # ### Reader
    #
    # A Reader scans the texts returned by retrievers in detail and extracts the k best answers. They are based
    # on powerful, but slower deep learning models.
    #
    # Haystack currently supports Readers based on the frameworks FARM and Transformers.
    # With both you can either load a local model or one from Hugging Face's model hub (https://huggingface.co/models).
    # **Here:** a medium sized RoBERTa QA model using a Reader based on
    #           FARM (https://huggingface.co/deepset/roberta-base-squad2)
    # **Alternatives (Reader):** TransformersReader (leveraging the `pipeline` of the Transformers package)
    # **Alternatives (Models):** e.g. "distilbert-base-uncased-distilled-squad" (fast) or
    #                            "deepset/bert-large-uncased-whole-word-masking-squad2" (good accuracy)
    # **Hint:** You can adjust the model to return "no answer possible" with the no_ans_boost. Higher values mean
    #           the model prefers "no answer possible"
    #
    # #### FARMReader

    # Load a  local model or any of the QA models on
    # Hugging Face's model hub (https://huggingface.co/models)
    #reader = FARMReader(model_name_or_path="deepset/xlm-roberta-large-squad2", use_gpu=True)

    # #### TransformersReader

    # Alternative:
    # reader = TransformersReader(
    #     model_name_or_path="distilbert-base-uncased-distilled-squad", tokenizer="distilbert-base-uncased",
    #     use_gpu=-1)

    # ### Pipeline
    # 
    # With a Haystack `Pipeline` you can stick together your building blocks to a search pipeline.
    # Under the hood, `Pipelines` are Directed Acyclic Graphs (DAGs) that you can easily customize for your own use cases.
    # To speed things up, Haystack also comes with a few predefined Pipelines. One of them is the `ExtractiveQAPipeline` that combines a retriever and a reader to answer our questions.
    # You can learn more about `Pipelines` in the [docs](https://haystack.deepset.ai/docs/latest/pipelinesmd).
    from haystack.pipeline import ExtractiveQAPipeline
    #from haystack.pipeline import FAQPipeline


    pipe = ExtractiveQAPipeline(reader, retriever)

    ## Voilà! Ask a question!
    # prediction = pipe.run(
    #     query="Who is the father of Arya Stark?"
    # )

    # questions = ['Who is in the board of directors?',
    #              'What is the nature of business?',
    #              'When the company was started?',
    #              'Where the company operates?',
    #              'How many workers they have?']


    questions_en = ['What is the net debt?',
                    'Wie hoch ist der Umsatz in der Gruppe?',
                    'What is the revenue in group?',
                    'What is the net profit?']


    questions_th = ['หนี้สุทธิคืออะไร??',
                    'รายได้ในกลุ่มคืออะไร?',
                    'กำไรสุทธิคืออะไร?',
                    'how does the covid-19 affect the company?']
    
    questions_de = ['Wie hoch ist die Nettoverschuldung? ',
                    'Wie hoch ist der Umsatz in der Gruppe?' ]


    questions_fr = ['Quelle est la dette nette?',
                    'Quel est le revenu du groupe?',
                    'Quel est le chiffre daffaires du groupe? ']


    questions_es = ['¿Cuáles son el resultado de la crisis?',
                 '¿Cuál es la naturaleza del negocio?',
                 '¿Cuándo se fundó la empresa?',
                 ]

    def predict(questions, pipe):
      for question in questions:
          prediction = pipe.run(
              query=question
          )
          print(prediction['query'])
          print(prediction['answers'][0]['answer'])
          print(prediction['answers'][0]['probability'])
          print(prediction['answers'][0]['context'])

    predict(questions_en, pipe)

    #prediction = pipe.run(query="Who created the Dothraki vocabulary?")
    #prediction = pipe.run(query="Who is the sister of Sansa?")

    # Now you can either print the object directly

    # Sample output:    
    # {
    #     'answers': [ <Answer: answer='Eddard', type='extractive', score=0.9919578731060028, offsets_in_document=[{'start': 608, 'end': 615}], offsets_in_context=[{'start': 72, 'end': 79}], document_id='cc75f739897ecbf8c14657b13dda890e', meta={'name': '454_Music_of_Game_of_Thrones.txt'}}, context='...' >,
    #                  <Answer: answer='Ned', type='extractive', score=0.9767240881919861, offsets_in_document=[{'start': 3687, 'end': 3801}], offsets_in_context=[{'start': 18, 'end': 132}], document_id='9acf17ec9083c4022f69eb4a37187080', meta={'name': '454_Music_of_Game_of_Thrones.txt'}}, context='...' >,
    #                  ...
    #                ]
    #     'documents': [ <Document: content_type='text', score=0.8034909798951382, meta={'name': '332_Sansa_Stark.txt'}, embedding=None, id=d1f36ec7170e4c46cde65787fe125dfe', content='\n===\'\'A Game of Thrones\'\'===\nSansa Stark begins the novel by being betrothed to Crown ...'>,
    #                    <Document: content_type='text', score=0.8002150354529785, meta={'name': '191_Gendry.txt'}, embedding=None, id='dd4e070a22896afa81748d6510006d2', 'content='\n===Season 2===\nGendry travels North with Yoren and other Night's Watch recruits, including Arya ...'>,
    #                    ...
    #                  ],
    #     'no_ans_gap':  11.688868522644043,
    #     'node_id': 'Reader',
    #     'params': {'Reader': {'top_k': 5}, 'Retriever': {'top_k': 5}},
    #     'query': 'Who is the father of Arya Stark?',
    #     'root_node': 'Query'
    # }

    # Note that the documents contained in the above object are the documents filtered by the Retriever from
    # the document store. Although the answers were extracted from these documents, it's possible that many
    # answers were taken from a single one of them, and that some of the documents were not source of any answer.

    # Or use a util to simplify the output
    # Change `minimum` to `medium` or `all` to raise the level of detail
    # print("\n\nSimplified output:\n")
    # print(prediction)
    # print_answers(prediction, details="minimum")


if __name__ == "__main__":
    tutorial1_basic_qa_pipeline()

# This Haystack script was made with love by deepset in Berlin, Germany
# Haystack: https://github.com/deepset-ai/haystack
# deepset: https://deepset.ai/


Vorwort

Der Halbjahresbericht enthält bestimmte finanzielle

Leistungskennzahlen, die nicht nach IFRS definiert sind und

die von der Geschäftsleitung dazu verwendet werden, die fi­

nanzielle und die operative Leistung der Gruppe zu bewerten.

Dazu zählen unter anderem:

– das organische Wachstum, das interne Realwachstum

und Preisanpassungen;

– die zugrunde liegende operative Ergebnismarge und die

operative Ergebnismarge;

– die Nettoverschuldung;

– der freie Geldfluss; und

– der zugrunde liegende Gewinn je Aktie (Earnings per

Share EPS) und der EPS bei konstanten Wechselkursen.

Die Geschäftsleitung geht davon aus, dass diese nicht nach

IFRS definierten finanziellen Leistungskennzahlen hilfreiche

Informationen zur finanziellen und operativen Leistung der

Gruppe liefern.

Im Dokument «Alternative Leistungskennzahlen», das unter

www.nestle.com/investors/publications veröffentlicht ist,

werden diese nicht nach IFRS definierten finanziellen

Leistungskennzahlen definiert.

H

/usr/local/lib/python3.7/dist-packages/haystack/document_store/sql.py:294: SAWarning: relationship 'DocumentORM.Meta' will copy column document.id to column meta.document_id, which conflicts with relationship(s): 'DocumentORM.meta' (copies document.id to meta.document_id), 'MetaORM.Document' (copies document.id to meta.document_id). If this is not the intention, consider if these relationships should be linked with back_populates, or if viewonly=True should be applied to one or more if they are read-only. For the less common case that foreign key constraints are partially overlapping, the orm.foreign() annotation can be used to isolate the columns that should be written towards.   To silence this warning, add the parameter 'overlaps="Document,meta"' to the 'DocumentORM.Meta' relationship. (Background on this error at: https://sqlalche.me/e/14/qzyx)
  meta_orms = [MetaORM(name=key, value=value) for key, value in meta_fields.items()]
/usr/local/lib/python3.7/dist-packages/haystack/docume

What is the net debt?
CHF 38,5 Milliarden
0.9298259615898132
overschuldung

Die Nettoverschuldung stieg per 30. Juni 2021 auf

CHF 38,5 Milliarden, verglichen mit CHF 31,3 Milliarden am

31. Dezember 2020. Der A


Inferencing Samples: 100%|██████████| 1/1 [00:40<00:00, 40.27s/ Batches]


Wie hoch ist der Umsatz in der Gruppe?
CHF 1,4 Milliarden
0.1571597009897232
erzeichneten ein Wachstum

von 16,7% und der Umsatz kletterte auf CHF 1,4 Milliarden in

79 Märkten. Purina­Produkte für Heimtiere erzielten ein zwei­


Inferencing Samples: 100%|██████████| 1/1 [00:39<00:00, 39.78s/ Batches]


What is the revenue in group?
Umsatz der Gruppe

Das organische Wachstum erreichte 8,1%
0.03432765230536461


und den Geschäftsmix.

Resultate der Gruppe

Umsatz der Gruppe

Das organische Wachstum erreichte 8,1% und das interne

Realwachstum betrug 6,8%. Di


Inferencing Samples: 100%|██████████| 1/1 [00:40<00:00, 40.02s/ Batches]

What is the net profit?
CHF 7,0 Milliarden
0.35945063829421997
lage­

vermögen. Das operative Ergebnis erhöhte sich um 0,2% auf

CHF 7,0 Milliarden. Die zugrunde liegende operative Ergebnis­

marge belief sich auf


In [ ]:
!pip freeze | grep farm

farm==0.7.1
farm-haystack==0.8.0


# Nowa sekcja

total 56
drwxr-xr-x 4 root root 4096 Jan 19 10:44 mlruns
-rw-r--r-- 1 root root 9570 Jan 19 11:36 nestle_report_de.txt
-rw-r--r-- 1 root root 8625 Jan 19 11:36 nestle_report_en.txt
-rw-r--r-- 1 root root 9944 Jan 19 11:36 nestle_report_fr.txt
drwxr-xr-x 1 root root 4096 Jan  7 14:33 sample_data
-rw-r--r-- 1 root root 2831 Jan 19 10:01 spanish_report.txt
-rw-r--r-- 1 root root 1653 Jan 19 10:01 telefonica_report_en.txt
-rw-r--r-- 1 root root 1892 Jan 19 10:01 telefonica_report_es.txt
